In [21]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from app.data_processing.utils import *

In [2]:
import pandas as pd
import pattern.es as es
import regex as re

In [13]:
fr = pd.read_csv('../data/peso_frutas.csv')
vr = pd.read_csv('../data/peso_hortalizas.csv')
bedca = pd.read_csv('../data/bedca.csv',index_col=0)

In [24]:
bedca_ingredients = bedca["nombre"].tolist()
bedca_ingredients = [x.lower() for x in bedca_ingredients]
bedca_ingredients = ingredients_to_plural(bedca_ingredients) 

In [14]:
fr = fr.drop('Peso bruto (g)',axis=1)

In [15]:
vg = fr.append(vr)

In [16]:
vg['Alimento'] = vg['Alimento'].apply(es.singularize)

In [22]:
def get_numeric_value(row):
    numbers = [word for word in row.split() if word.isdigit()]
    if len(numbers) > 0:
        return int(numbers[0])
    return None

def set_bedca_name(row):
    ext = extract_ingredient(row,bedca_ingredients)
    if ext != "None":
        return ext
    else:
        return row

In [25]:
vg['Numero'] = vg['Medida'].apply(get_numeric_value)
vg['Alimento'] = vg['Alimento'].apply(set_bedca_name)
vg['Peso x Unidad'] = vg['Peso neto (g)'].divide(vg['Numero'])
vg.rename(columns={'Alimento': 'Ingrediente'}, inplace=True)

In [12]:
vg.to_csv('../data/pesos_vegetales.csv', index=False)